# Drug DB Tutorial

This tutorial will go through how to
#1) Download and reload the Chembl postgresql database 
#2) Extract protein-ligand affinity information from the ChEMBL database uisng psycopg2
#3) Generate a multigraph database using networkx


In [ ]:
import psycopg2 
import networkx as nx
import db_network as db 

# Step 1) 

Download the postgresql Chembl_24 database from here https://www.ebi.ac.uk/chembl/downloads

The total size of the database is ~1.2 GB so it may take some time. Once downloaded, you can restore the database by following the instructions on included with the tarfile. This will take >30 minutes.


# Step 2)

We use psycopg2, a python-based postgresql adapter to communicate with the database and extract the information we are interested in.

Unfortunately, the Chembl_24 is very complex. You can take a look at the schema here ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_24_1_schema.png

For this tutorial, we will retrieve all the molecules with a pchembl affinity of > 2 associated with the 107 proteins below


The syntax for the retrieval is normal sql, you can take a look at the schema documentation for each table here ftp://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_24_1_schema_documentation.html

In [1]:
protein_list=['P31749', 'P34998', 'P31213', 'P34995', 'Q86WA6', 'P33527', 'O15075', 'P16435', 'P30613', 'P84077', 'P23919', 'Q9BUF5', 'Q00535', 'Q00534', 'Q00537', 'Q00536', 'P13100', 'O95259', 'O95255', 'Q7DHH4', 'Q9UBM7', 'O92972', 'P21673', 'P30291', 'P07711', 'P17050', 'Q07699', 'P40429', 'P0AB71', 'P62258', 'P08842', 'P03891', 'P9WKD7', 'P03897', 'P02768', 'P78348', 'P28593', 'P02763', 'P02766', 'P10275', 'P10276', 'Q9Y2U5', 'P20618', 'P52270', 'P09172', 'Q08188', 'O00408', 'P20813', 'P29401', 'P07510', 'P35247', 'O75438', 'P01127', 'P08620', 'Q01118', 'P17443', 'P9WIJ3', 'P01031', 'Q9UK17', 'P02749', 'Q92831', 'P17931', 'P17936', 'P9WI73', 'Q9A1X4', 'P06280', 'Q99884', 'Q9Y6E0', 'P01019', 'P01011', 'Q99640', 'O43570', 'O75385', 'O75380', 'P04036', 'P04035', 'O75899', 'Q9BXA5', 'Q9BXA7', 'P00722', 'P00720', 'O60760', 'P62714', 'Q05901', 'Q15185', 'P20035', 'Q99661', 'Q9NRH2', 'Q05586', 'P25774', 'Q09013', 'P54284', 'P25779', 'O75762', 'O43246', 'Q9NRF9', 'P43490', 'P15428', 'Q96Q83', 'P27448', 'Q9Y6I3', 'P16152', 'Q9HCR9', 'Q16850', 'Q16853', 'P99999', 'P04626']

In [ ]:
conn = psycopg2.connect("dbname='CHEMBL_24' user='_USERNAME_' host='_HOST_' password='_PASSWORD_'")   ## connect to the database
curr = conn.cursor()   ## create a cursor object

In [ ]:
drug_affinity_uni={}      ### create a temporary dict with the information
for ux,u in enumerate(protein_list):
    curr.execute(" SELECT md.chembl_id, act.PCHEMBL_VALUE , cs.canonical_smiles, md.PREF_NAME  \
                 FROM TARGET_COMPONENTS tc JOIN COMPONENT_SEQUENCES cq ON cq.COMPONENT_ID = tc.COMPONENT_ID \
                 JOIN target_dictionary td ON td.tid = tc.tid  \
                 JOIN assays a ON td.tid = a.tid \
                 JOIN activities act ON a.assay_id = act.assay_id \
                 JOIN molecule_dictionary md ON md.molregno = act.molregno \
                 JOIN compound_structures cs ON md.molregno = cs.molregno \
                 AND cq.ACCESSION = '"+str(u)+"' \
                 AND act.standard_relation = '=' AND act.PCHEMBL_VALUE > 2 ")            
    drug_affinity_uni[u]=curr.fetchall()


# Step 3)
Once we have retrieved the protein-ligand affinity information we are interested in, we save this information in the form of a networkx MultiGraph which we can pickle and load into a db_network object

In [ ]:
mgraph = nx.MultiGraph()

chembl_list = set()
for p in protein_list:
    for ch in [s[0] for s in drug_affinity_uni[p]]:
        chembl_list.add(ch)


In [ ]:
## populate the mgraph object with the protein target nodes from the protein_list and the drug nodes from the chembl_list

for p in protein_list:
    mgraph.add_node(p)
for ch in chembl_list:
    mgraph.add_node(ch)

In [ ]:
## Create the protein-drug edges and label the weight property of the edge with the affinity of the interaction

for p in protein_list:
    for s in drug_affinity_uni[p]:
        ch=s[0]
        aff=float(s[1])
        mgraph.add_edge(p,ch,weight=aff)

In [ ]:
## store the graph object
db.pickle('DB_multigraph.obj',mgraph)

In [ ]:
## load the multigraph into db_net object

db_net = db.db_network(mgraph) 